# Google Analytics Kaggle Challenge

## Import the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from sklearn import linear_model
import itertools as it

## Define variables

In [2]:
data_path = 'D:/000_Projects_2018/0002_Development/Kaggle/google_analytics/data/'

## Define functions

In [3]:
def separate_json(series: pd.Series) -> pd.DataFrame():
    """
    
    Args:
        series: Series before json parsing 

    Returns: DataFrame

    """
    # TODO: Write TypeException
    
    if isinstance(series[0], str):
        return pd.DataFrame(json.loads(s) for s in series)
    return pd.DataFrame(s for s in series)

## Extract Transform Load: ETL

In [4]:
json_col = ['device', 'geoNetwork', 'totals', 'trafficSource']
nest_json_col = ['adwordsClickInfo']

df_train = pd.read_csv('{}train.csv'.format(data_path), engine='python')
df_train = df_train.join(separate_json(
    df_train[col_name]) for col_name in json_col).drop(json_col, axis=1)
df_train = df_train.join(separate_json(
    df_train[nest_json_col[0]])).drop(nest_json_col, axis=1)

df_test = pd.read_csv('{}test.csv'.format(data_path), engine='python')
df_test = df_test.join(separate_json(
    df_test[col_name]) for col_name in json_col).drop(json_col, axis=1)
df_test = df_test.join(separate_json(
    df_test[nest_json_col[0]])).drop(nest_json_col, axis=1)

In [5]:
print("The train dataset has {} columns".format(len(df_train.columns)))
print("The test dataset has {} columns".format(len(df_test.columns)))
print("The number of rows in the train dataset:{}".format(len(df_train)))
print("The number of rows in the test dataset:{}".format(len(df_test)))

The train dataset has 56 columns
The test dataset has 54 columns
The number of rows in the train dataset:903653
The number of rows in the test dataset:804684


In [6]:
#df_train_naidu = pd.read_csv('{}extracted_fields_train.csv'.format(data_path), engine='python')
#df_test_naidu = pd.read_csv('{}extracted_fields_test.csv'.format(data_path), engine='python')

In [7]:
#print("The train dataset has {} columns".format(len(df_train_naidu.columns)))
#print("The test dataset has {} columns".format(len(df_test_naidu.columns)))
#print("The number of rows in the train dataset:{}".format(len(df_train_naidu)))
#print("The number of rows in the test dataset:{}".format(len(df_test_naidu)))

In [8]:
for column in list(df_train.columns):
    if column not in list(df_test.columns):
        print(column)

transactionRevenue
campaignCode


In [9]:
def f4(x):
    print("x: column nr.{} in the training set which is {}".format(x,list(df_train.columns)[x]))
    print(df_train[list(df_train.columns)[x]].value_counts(dropna=False))

interact(f4, x=widgets.IntSlider(min=0,max=len(df_train.columns)-1,step=1,value=0));


interactive(children=(IntSlider(value=0, description='x', max=55), Output()), _dom_classes=('widget-interact',))

In [10]:
def f5(x):
    print("x: column nr.{} in the test set which is {}".format(x,list(df_test.columns)[x]))
    print(df_test[list(df_test.columns)[x]].value_counts(dropna=False))

interact(f5, x=widgets.IntSlider(min=0,max=len(df_test.columns)-1,step=1,value=0));

interactive(children=(IntSlider(value=0, description='x', max=53), Output()), _dom_classes=('widget-interact',))

## Data aggregation

In [11]:
print(len(df_train.groupby(['fullVisitorId']).sum()))
print(len(df_test.groupby(['fullVisitorId']).sum()))

714167
617242


In [12]:
fullVisitorId_test = df_test.fullVisitorId

In [13]:
expected_column_to_detet=['fullVisitorId','socialEngagementType','browserSize','browserVersion','flashVersion',
                         'language','mobileDeviceBranding','mobileDeviceInfo','mobileDeviceMarketingName',
                         'mobileDeviceModel','mobileInputSelector','operatingSystemVersion','screenColors',
                          'screenResolution','cityId','latitude','longitude','networkLocation','visits',
                          'campaignCode','criteriaParameters','targetingCriteria']

In [14]:
def drop_unnecessary_columns(expected_column_to_detet,df_train,df_test):
    for col in expected_column_to_detet:
        df_train = df_train.drop([col],axis = 1)
    for col in expected_column_to_detet:
        try:
            df_test = df_test.drop([col],axis = 1)
        except:
            print('Column {} is not exiting in the test dataset'.format(col))
    return df_train, df_test

In [15]:
df_train, df_test = drop_unnecessary_columns(expected_column_to_detet,df_train,df_test)

Column campaignCode is not exiting in the test dataset


In [16]:
print("The clean train dataset has {} columns".format(len(df_train.columns)))
print("The clean test dataset has {} columns".format(len(df_test.columns)))

The clean train dataset has 34 columns
The clean test dataset has 33 columns


In [17]:
train_revenue = df_train[~df_train['transactionRevenue'].isnull()].copy()
print(len(train_revenue))

11515


In [18]:
try:
    Y_train = df_train.transactionRevenue
    df_train = df_train.drop(['transactionRevenue'],axis = 1)
except:
    print('the target already deleted')

## Replace the strings with integers

In [22]:
def replace_strings_integer(df_train, df_test):
    df_total = pd.concat([df_train,df_test],sort=False)
    df_total.index=range(len(df_total['date']))
    df_train_decoded = df_train
    df_test_decoded= df_test
    for col_i in df_train.columns[df_train.dtypes == 'object']:
            
        df_total[col_i] = df_total[col_i].factorize()[0]
        df_train_decoded[col_i] = df_total.loc[range(df_train.shape[0]),col_i].values
        df_test_decoded[col_i] =  df_total.loc[range(df_train.shape[0],
                                                     df_train.shape[0]+df_test.shape[0]),
                                               col_i].values
    return df_train_decoded, df_test_decoded

In [23]:
df_train, df_test = replace_strings_integer(df_train, df_test)

In [24]:
Y_train = Y_train.fillna(0)
print(Y_train.value_counts(dropna=False))

0            892138
16990000        256
18990000        189
33590000        187
44790000        170
13590000        135
55990000        122
19990000        116
15990000         98
15190000         93
19190000         92
10990000         84
59990000         81
24990000         77
79990000         65
27190000         64
27180000         62
33980000         54
39990000         51
67180000         46
1990000          44
28780000         40
21990000         40
30390000         39
35980000         39
17590000         38
27980000         37
12990000         37
30380000         37
31990000         37
              ...  
122360000         1
512370000         1
478480000         1
166480000         1
110320000         1
531620000         1
201460000         1
770000            1
215380000         1
82240000          1
120890000         1
30260000          1
11700000          1
300850000         1
37940000          1
166950000         1
44660000          1
223200000         1
136750000         1


In [25]:
train_revenue = Y_train[Y_train.astype('int64')>0].copy()
min(train_revenue)
np.log1p(min(train_revenue.astype('int64')))

9.2104403669765169

In [26]:
## interactive tool
col_train = df_train.columns[df_train.dtypes != 'object']
col_test = df_test.columns[df_test.dtypes != 'object']
print(len(col_train))
print(len(col_test))

def f(x,print_feature=False):
    
    compare_columns(x,df_train,df_test,print_feature)

33
33


In [27]:
df_corr_train = df_train.fillna(0).corr()
df_corr_test = df_test.fillna(0).corr()

In [28]:
def compare_columns(col_i,df_train,df_test,print_feature):
    
    col_train = df_train.columns[df_train.dtypes != 'object']
    col_test = df_test.columns[df_test.dtypes != 'object']

    print('the column {} in both training and testing sets are: '.format(col_i) + 
          col_train[col_i] +' , ' + col_test[col_i])
    print('the mean value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].mean()) +' , ' + str(df_test[col_test[col_i]].mean()))
    print('the std value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].std()) +' , ' + str(df_test[col_test[col_i]].std()))
    print('the max value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].max()) +' , ' + str(df_test[col_test[col_i]].max()))
    print('the min value of the column {} in both training and testing sets are: '.format(col_i) + 
          str(df_train[col_train[col_i]].min()) +' , ' + str(df_test[col_test[col_i]].min()))
    print('Number of uniqe values of the column {}: '.format(col_i) + 
          str(len(df_train[col_train[col_i]].value_counts())) + '/' + str(len(df_train[col_train[col_i]])) + ' , ' +
          str(len(df_test[col_test[col_i]].value_counts())) + '/' + str(len(df_test[col_test[col_i]])))
    print('Number of uniqe values of the column {} in both datasets compared to the uniqe values in train set: '.format(col_i) + 
          str(len(pd.concat([df_train[col_train[col_i]],df_test[col_test[col_i]]]).value_counts())) + '/' +
         str(len(df_train[col_train[col_i]].value_counts())))
    
    print('The most frequent values of the column {} in both datasets: \n'.format(col_i) + 
          str(df_train[col_train[col_i]].value_counts().head()) + ' , \n' +
         str(df_test[col_test[col_i]].value_counts().head()))
    print ('correlation between the feature and the target')
    print(np.corrcoef([df_train[col_train[col_i]].fillna(0),Y_train.astype('int64')]))
    plt.matshow(df_corr_train)
    
    plt.matshow(df_corr_test)
    if print_feature:
        
        plt.scatter(df_train[col_train[0]],df_train[col_train[col_i]],c=Y_train)
    
    print(df_corr_train.loc[df_corr_train[col_train[col_i]]>0.85,col_train[col_i]].head())
    print(df_corr_test.loc[df_corr_test[col_train[col_i]]>0.85,col_train[col_i]].head())

In [29]:
interact(f, x=widgets.IntSlider(min=0,max=32,step=1,value=0));

interactive(children=(IntSlider(value=0, description='x', max=32), Checkbox(value=False, description='print_feature'), Output()), _dom_classes=('widget-interact',))

## insights:
hists,pageviews have 15 % correlation with the target

In [30]:
#extra_columns_to_delete = ['visitStartTime']   

In [31]:
#df_train, df_test = drop_unnecessary_columns(extra_columns_to_delete,df_train,df_test)

In [32]:
#print("The clean train dataset has {} columns".format(len(df_train.columns)))
#print("The clean test dataset has {} columns".format(len(df_test.columns)))

In [33]:
Y_train_b = (Y_train.astype('int64') > 0)*1

In [34]:
print(Y_train_b.value_counts(dropna=False))

0    892138
1     11515
Name: transactionRevenue, dtype: int64


In [ ]:
#random_state = 0
#x_train, x_cv, y_train, y_cv= train_test_split(df_train,Y_train_b,
#                       test_size=0.1,stratify=Y_train_b,
#                       random_state=random_state)

In [ ]:
#print(y_train.value_counts(dropna=False))

In [ ]:
#print(y_cv.value_counts(dropna=False))

In [ ]:
#clf = RandomForestClassifier(n_estimators=200,max_depth=15,random_state=0,n_jobs=-1)
#clf.fit(x_train, y_train)
#print('train:',clf.score(x_train, y_train))

In [ ]:
#print('cross-validation:',clf.score(x_cv, y_cv))

## Apply classification on the test data:

In [ ]:
#Y_test_b = clf.predict(df_test)

In [ ]:
#print(sum(Y_test_b))
#print(len(Y_test_b))

In [ ]:
#Y_train_n = Y_train[Y_train.astype('int64')>0].astype('int64')
#df_train_n = df_train[Y_train.astype('int64')>0]

In [ ]:
#print(Y_train_n.value_counts(dropna=False))

In [ ]:
#def f_1(x):
#    print(df_train_n.columns[x])
#    print('original')
#    print(np.corrcoef([df_train_n[df_train_n.columns[x]],Y_train_n]))
#    print('log')
#    print(np.corrcoef([df_train_n[df_train_n.columns[x]]**2,np.log(Y_train_n)]))
    
#interact(f_1, x=widgets.IntSlider(min=0,max=31,step=1,value=0));

In [ ]:
#column_to_delete = ['bounces']

In [ ]:
#df_train_n, _ = drop_unnecessary_columns(column_to_delete,df_train_n,df_test)

## Normaize the data

In [ ]:
#df_train_n_n=(df_train_n-np.mean(df_train_n))/np.std(df_train_n)
#df_test_n

In [ ]:
#linreg = linear_model.LinearRegression()
#linreg.fit(df_train_n_n, Y_train_n)

In [ ]:
#print(linreg.score(df_train_n_n,Y_train_n))

In [ ]:
#plt.plot(df_train_n_n[df_train_n_n.columns[12]],np.log(Y_train_n),'.')

In [35]:
def map_features(X, map_degree,maped_fea):
    V=np.zeros((len(maped_fea),1))
    cor_f=pd.DataFrame(maped_fea)
    com_x_f=[]
    for i in range(2,map_degree+1):
        com_x=list(it.combinations_with_replacement(range(1,10), i))#(range(n_x), i))
        for j in range(len(com_x)):
            if com_x[j][0]!=com_x[j][1] or com_x[j][0]!=com_x[j][1]:
                V[:,0]= X[:,com_x[j][0]]*X[:,com_x[j][1]]
                cor_f['V']=V
                X=np.append(X.T,np.array(X[:,com_x[j][0]]*X[:,com_x[j][1]]).reshape(1,-1),axis=0).T
                com_x_f.append(com_x[j])
    return X,com_x_f

In [36]:
#x_train_map,com_x_f=map_features(np.array(df_train_n_n),2,Y_train_n)

In [37]:
#linreg.fit(x_train_map, Y_train_n)

In [38]:
#print(linreg.score(x_train_map,Y_train_n))

In [39]:
#x_train_m, x_cv_m, y_train_m, y_cv_m= train_test_split(x_train_map,Y_train_n,
#                       test_size=0.1,random_state=random_state)

In [40]:
import lightgbm as lgb 

lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 200, "learning_rate" : 0.01, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
#lgb_train = lgb.Dataset(x_train_m, label=np.log(y_train_m))
#lgb_val = lgb.Dataset(x_cv_m, label=np.log(y_cv_m))
#model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=150, verbose_eval=20)

In [41]:
random_state = 0
x_train, x_cv, y_train, y_cv= train_test_split(df_train,Y_train.astype('int64'),
                       test_size=0.1,stratify=Y_train_b,random_state=random_state)

In [42]:
x_train_map,com_x_f=map_features(np.array(x_train),2,y_train)

In [43]:
def map_features_test(X, com_x_f):
    com_x=com_x_f
    for j in range(len(com_x)):
        X=np.append(X.T,np.array(X[:,com_x[j][0]]*X[:,com_x[j][1]]).reshape(1,-1),axis=0).T
    return X

In [44]:
x_cv_map=map_features_test(np.array(x_cv), com_x_f)

In [45]:
lgb_train = lgb.Dataset(x_train_map, label=np.log1p(y_train))
lgb_val = lgb.Dataset(x_cv_map, label=np.log1p(y_cv))
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=150, verbose_eval=20)

Training until validation scores don't improve for 150 rounds.
[20]	valid_0's rmse: 1.90878
[40]	valid_0's rmse: 1.83538
[60]	valid_0's rmse: 1.78174
[80]	valid_0's rmse: 1.74382
[100]	valid_0's rmse: 1.71541
[120]	valid_0's rmse: 1.69518
[140]	valid_0's rmse: 1.68142
[160]	valid_0's rmse: 1.6695
[180]	valid_0's rmse: 1.66065
[200]	valid_0's rmse: 1.65394
[220]	valid_0's rmse: 1.64888
[240]	valid_0's rmse: 1.64557
[260]	valid_0's rmse: 1.64282
[280]	valid_0's rmse: 1.64041
[300]	valid_0's rmse: 1.63838
[320]	valid_0's rmse: 1.63633
[340]	valid_0's rmse: 1.63458
[360]	valid_0's rmse: 1.63338
[380]	valid_0's rmse: 1.63233
[400]	valid_0's rmse: 1.63178
[420]	valid_0's rmse: 1.63117
[440]	valid_0's rmse: 1.63074
[460]	valid_0's rmse: 1.63078
[480]	valid_0's rmse: 1.63063
[500]	valid_0's rmse: 1.63058
[520]	valid_0's rmse: 1.63044
[540]	valid_0's rmse: 1.63031
[560]	valid_0's rmse: 1.63034
[580]	valid_0's rmse: 1.63032
[600]	valid_0's rmse: 1.63046
[620]	valid_0's rmse: 1.63036
[640]	valid_

In [46]:
x_test_map=map_features_test(np.array(df_test), com_x_f)

In [47]:
x_test_map.shape

(804684, 69)

In [48]:
df_test.shape

(804684, 33)

In [49]:
preds = model.predict(x_test_map, num_iteration=model.best_iteration)

In [50]:
print(len(preds))

804684


In [51]:
pred_sub = pd.DataFrame(fullVisitorId_test)

In [52]:
#pred_sub = pd.DataFrame(columns=['fullVisitorId','PredictedLogRevenue'])
#pred_sub.fullVisitorId = fullVisitorId_test
pred_sub['PredictedLogRevenue'] = preds

In [53]:
x =pred_sub[pred_sub.PredictedLogRevenue < 5].index

In [54]:
pred_sub.loc[x,'PredictedLogRevenue']=0
pred_sub.head()

,fullVisitorId,PredictedLogRevenue
0,6167871330617112363,0.0
1,0643697640977915618,0.0
2,6059383810968229466,0.0
3,2376720078563423631,0.0
4,2314544520795440038,0.0


In [63]:
#pred_sub.to_csv(data_path+'predict.csv',index=False)

In [ ]:
#pred_sub_agg = pred_sub.groupby(['fullVisitorId']).sum()

In [ ]:
#pred_sub_agg.head()

In [ ]:
#print(len(pred_sub_agg))

In [ ]:
#pred_sub_agg.to_csv('{}initial_submission_5.csv'.format(data_path))